In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q datasets

In [18]:
import json
import random
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader

In [19]:
f = open("/content/climate_dev_processed.json")
climate_dev = json.load(f)

In [20]:
for claim in climate_dev:
  if claim['label']=='REFUTED':
    claim['label'] = 0
  if claim['label']=='SUPPORTED':
    claim['label'] = 1
  if claim['label']=='NEI':
    claim['label'] = 2

In [21]:
claim_list = []
evidence_list = []
label_list = []
for claim in climate_dev:
  claim_list.append(claim['claim'])
  evidence_list.append(claim['context'])
  label_list.append(claim['label'])

test_dict = {'claim':claim_list, 'evidence':evidence_list, 'label':label_list}

In [22]:
test_dataset = Dataset.from_dict(test_dict)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("amandakonet/climatebert-fact-checking")

def tokenize_function(examples):
  return tokenizer(examples["claim"],examples["evidence"],padding="max_length", truncation=True)

test_tokenized_datasets = test_dataset.map(tokenize_function, batched=True)

In [24]:
test_tokenized_datasets = test_tokenized_datasets.remove_columns(["claim"])
test_tokenized_datasets = test_tokenized_datasets.remove_columns(["evidence"])
test_tokenized_datasets = test_tokenized_datasets.rename_column("label", "labels")
test_tokenized_datasets.set_format("torch")

In [28]:
eval_dataloader = DataLoader(test_tokenized_datasets, batch_size=16)

In [29]:
model = AutoModelForSequenceClassification.from_pretrained("amandakonet/climatebert-fact-checking")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

In [ ]:
model.eval()
eval_labels = []
with torch.no_grad():
   for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch).logits
   
        label_mapping = ['SUPPORTED', 'REFUTED', 'NEI']
        labels = [label_mapping[score_max] for score_max in outputs.argmax(dim=1)]
        eval_labels += labels
        print(labels)

In [31]:
labels = []
for claim in climate_dev:
  labels.append(claim['label'])

In [41]:
pred_labels = []
for label in eval_labels:
  if label=='NEI':
    pred_labels.append(2)
  if label=='SUPPORTED':
    pred_labels.append(0)
  if label=='REFUTED':
    pred_labels.append(1)



In [ ]:
correct_idx = []
correct = 0
for i in range(len(labels)):
  if pred_labels[i]!=2 and pred_labels[i]==labels[i]:
    correct_idx.append(i)
    correct += 1
print(correct/len(labels))

In [45]:
print(correct_idx)

[1, 5, 7, 12, 15, 19, 25, 31, 33, 46, 48, 50, 56, 57, 60, 63, 64, 65, 72, 74, 75, 78, 79, 80, 82, 85, 87, 88, 94, 97, 102, 103, 104, 105, 114, 118, 121, 125, 127, 128, 134, 146, 147, 160, 161, 163, 168, 169, 170, 174, 175, 183, 185, 186, 190, 200, 204, 211, 212, 217, 226, 230, 236, 246, 247, 253, 257, 258, 263, 264, 270, 279, 282, 283, 290, 296, 308, 310, 314, 316, 319, 334, 338, 340, 346, 348, 349, 352, 354, 362, 364, 366, 369, 370, 376, 378, 379, 380, 381, 392, 405, 410, 429, 430, 436, 438, 440, 445, 449, 452, 456, 458, 465, 466, 474, 478, 485, 491, 503, 509, 514, 518, 532, 535, 538, 540, 542, 544, 545, 547, 554, 561, 565, 572, 574, 575, 576, 582, 583, 585, 586, 587, 594, 595, 603, 608, 610, 626, 633, 641, 642, 648, 652, 654, 655, 657, 660, 666, 667, 669, 673, 676, 687, 688, 697, 699, 705, 709, 717, 724, 731, 740, 746, 757, 758, 772, 774, 784, 792, 795, 796, 803, 805, 811, 813, 815, 820, 822, 825, 826, 831, 834, 843, 846, 849, 850, 852, 858, 861, 864, 869, 870, 873, 874, 879, 883, 88